<a href="https://colab.research.google.com/github/dogukartal/IBM_AI_Labs/blob/main/Generative%20AI%20Language%20Modeling%20with%20Transformers/Pre_training_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pretraining BERT Models**
---


## Setup


In [ ]:
!pip install -qq torch==2.1.0
!pip install -Uqq portalocker>=2.0.0
!pip install -qq torchtext==0.16.0

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import Transformer
from transformers import BertTokenizer
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchtext.vocab import Vocab,build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import IMDB
import random
from itertools import chain
import pandas as pd
from copy import deepcopy
import csv
import json
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import get_linear_schedule_with_warmup

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


## Background

### Pretraining Objectives

1. Masked Language Modeling (MLM):
   Masked language modeling involves randomly masking some words in a sentence and training the model to predict the masked words based on the context provided by the surrounding words(i.e., words that appear either before or after the masked word). The objective is to enable the model to learn contextual understanding and fill in missing information.

   Here's how MLM works:
   - Given an input sentence, a certain percentage of the words are randomly chosen and replaced with a special [MASK] token.
   - The model's task is to predict the original words that were masked, given the context of the surrounding words.
   - During training, the model learns to understand the relationship between the masked words and the rest of the sentence, effectively capturing the contextual information.

2. Next Sentence Prediction (NSP):
   Next sentence prediction involves training the model to predict whether two sentences are consecutive in the original text or randomly chosen from the corpus. This objective helps the model learn sentence-level relationships and understand the coherence between sentences.

   Here's how NSP works:
   - Given a pair of sentences, the model is trained to predict whether the second sentence follows the first sentence in the original text or if it is randomly selected from the corpus.
   - The model learns to capture the relationships between sentences and understand the flow of information in the text.

   NSP is particularly useful for tasks that involve understanding the relationship between multiple sentences, such as question answering or document classification. By training the model to predict the coherence of sentence pairs, it learns to capture the semantic connections between them.

It's important to note that different pretrained models may use variations or combinations of these objectives, depending on the specific architecture and training setup.


## Loading Data


In [ ]:
!wget -O BERT_dataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bZaoQD52DcMpE7-kxwAG8A.zip
!unzip BERT_dataset.zip

In [ ]:
# Create a torch Dataset using the CSV file
class BERTCSVDataset(Dataset):
    def __init__(self, filename):
        self.data = pd.read_csv(filename)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            bert_input = torch.tensor(json.loads(row['BERT Input']), dtype=torch.long)
            bert_label = torch.tensor(json.loads(row['BERT Label']), dtype=torch.long)
            segment_label = torch.tensor([int(x) for x in row['Segment Label'].split(',')], dtype=torch.long)
            is_next = torch.tensor(row['Is Next'], dtype=torch.long)
            original_text = row['Original Text']  # If you want to use it
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for row {idx}: {e}")
            print("BERT Input:", row['BERT Input'])
            print("BERT Label:", row['BERT Label'])
            # Handle the error
            return None  # or some default values
        return bert_input, bert_label, segment_label, is_next  # Include original_text if needed

In [ ]:
PAD_IDX = 0

def collate_batch(batch):
    bert_inputs_batch, bert_labels_batch, segment_labels_batch, is_nexts_batch = [], [], [], []

    for bert_input, bert_label, segment_label, is_next in batch:
        # Convert each sequence to a tensor and append to the respective list
        bert_inputs_batch.append(torch.tensor(bert_input, dtype=torch.long))
        bert_labels_batch.append(torch.tensor(bert_label, dtype=torch.long))
        segment_labels_batch.append(torch.tensor(segment_label, dtype=torch.long))
        is_nexts_batch.append(is_next)

    # Pad the sequences in the batch
    bert_inputs_final = pad_sequence(bert_inputs_batch, padding_value=PAD_IDX, batch_first=False)
    bert_labels_final = pad_sequence(bert_labels_batch, padding_value=PAD_IDX, batch_first=False)
    segment_labels_final = pad_sequence(segment_labels_batch, padding_value=PAD_IDX, batch_first=False)
    is_nexts_batch = torch.tensor(is_nexts_batch, dtype=torch.long)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_batch

In [ ]:
BATCH_SIZE = 2

train_dataset_path = './bert_dataset/bert_train_data.csv'
test_dataset_path = './bert_dataset/bert_test_data.csv'

train_dataset = BERTCSVDataset(train_dataset_path)
test_dataset = BERTCSVDataset(test_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

## Model creation

In BERT, there are three types of embedings

1. Token Embedding: Token embedding is the initial representation of each token in a BERT model. It maps each token to a dense vector representation of a fixed size, typically referred to as the embedding size. The token embedding layer in BERT learns the contextual representations of the input tokens. These embeddings capture the semantic meaning of the tokens and their relationships with other tokens in the context.

2. Positional Embedding: BERT is a transformer-based model that processes the input tokens in parallel. However, since transformers don't inherently capture the order of tokens, positional embedding is used to inject positional information into the model. It adds a vector representation to each token that encodes its position in the input sequence. The positional embedding allows BERT to understand the sequential order of the tokens and capture their relative positions.

3. Segment Embedding: BERT can handle sentence pairs or sequences that have distinct segments or parts. To differentiate between different segments, such as sentences or document sections, segment embedding is used. It assigns a unique vector representation to each segment or part of the input. The segment embeddings help BERT understand the relationships between different segments and capture the context within and between them.


In [ ]:
EMBEDDING_DIM = 10

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        # Create a positional encoding matrix as per the Transformer paper's formula
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        # Apply the positional encodings to the input token embeddings
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [ ]:
class BERTEmbedding (nn.Module):

    def __init__(self, vocab_size, emb_size, dropout=0.1, train=True):
        super().__init__()
        self.token_embedding = TokenEmbedding(vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout)
        self.segment_embedding = nn.Embedding(3, emb_size)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, bert_inputs, segment_labels=False):
        my_embeddings=self.token_embedding(bert_inputs)
        if self.train:
          x = self.dropout(my_embeddings + self.positional_encoding(my_embeddings) + self.segment_embedding(segment_labels))
        else:
          x = my_embeddings + self.positional_encoding(my_embeddings)

        return x

In [ ]:
VOCAB_SIZE=147161
batch = 2
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load sample batches from dataloader
for batch in train_dataloader:
  bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]
  count += 1
  if count == 5:
      break

In [ ]:
# bert_inputs, bert_labels, segment_labels, is_nexts = next(iter(train_dataloader))

In [ ]:
bert_inputs.shape

torch.Size([100, 3])

In [ ]:
bert_inputs[:,0]

tensor([    1,     5,     3,    64, 20802,    43,   334,   121,     3,    64,
           20,    78,    44,   226,  1221,  1741,    11,    14,     6,     2,
            0,    64,  1258,    38,    10,     3,   101,  7953,  1268,  1886,
         1209,  8608,   134,  1069,    25,    49,    20,  1747,    41,   354,
            6,     2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       device='cuda:0')

In [ ]:
segment_labels.shape

torch.Size([100, 3])

In [ ]:
segment_labels[:,0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')

In [ ]:
# Instantiate the TokenEmbedding
token_embedding = TokenEmbedding(VOCAB_SIZE, emb_size=EMBEDDING_DIM).to(device)

# Get the token embeddings for a sample input
t_embeddings = token_embedding(bert_inputs)

#Each token is transformed into a tensor of size emb_size
print(f"Dimensions of token embeddings: {t_embeddings.size()} \n") # Expected: (sequence_length, batch_size, EMBEDDING_DIM)

#Check the embedded vectors for first 3 tokens of the first sample in the batch
# you get embeddings[i,0,:] where i refers to the i'th token of the first sample in the batch (b=0)
for i in range(3):
    print(f"Token Embeddings for the {i}th token of the first sample: {t_embeddings[i,0,:]}")

Dimensions of token embeddings: torch.Size([100, 3, 10]) 

Token Embeddings for the 0th token of the first sample: tensor([ 0.9975,  2.7258, -2.7573,  3.3481,  2.5082,  1.8394, -0.5717, -1.3693,
        -4.1012, -0.3087], device='cuda:0', grad_fn=<SliceBackward0>)
Token Embeddings for the 1th token of the first sample: tensor([-4.4976,  2.6946, -6.5365, -4.9356, -0.0905,  1.2110,  0.0913,  0.7686,
        -3.9067,  1.4908], device='cuda:0', grad_fn=<SliceBackward0>)
Token Embeddings for the 2th token of the first sample: tensor([-3.1621, -0.4848, -6.7954,  6.0018,  1.2483,  0.8448, -1.7277, -2.5739,
        -3.6704,  1.4464], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
positional_encoding = PositionalEncoding(emb_size=EMBEDDING_DIM,dropout=0).to(device)

# Apply positional encoding to token embeddings
p_embedding = positional_encoding(t_embeddings)

print(f"Dimensions of positionally encoded tokens: {p_embedding.size()}")# Expected: (sequence_length, batch_size, EMBEDDING_DIM)

#Check the positional encoded vectors for first 3 tokens of the first sample in the batch
# you get encoded_tokens[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"Positional Embeddings for the {i}th token of the first sample: {p_embedding[i,0,:]}")

Dimensions of positionally encoded tokens: torch.Size([100, 3, 10])
Positional Embeddings for the 0th token of the first sample: tensor([ 0.9975,  3.7258, -2.7573,  4.3481,  2.5082,  2.8394, -0.5717, -0.3693,
        -4.1012,  0.6913], device='cuda:0', grad_fn=<SliceBackward0>)
Positional Embeddings for the 1th token of the first sample: tensor([-3.6561,  3.2349, -6.3786, -3.9482, -0.0653,  2.2107,  0.0953,  1.7686,
        -3.9061,  2.4908], device='cuda:0', grad_fn=<SliceBackward0>)
Positional Embeddings for the 2th token of the first sample: tensor([-2.2528, -0.9009, -6.4837,  6.9520,  1.2985,  1.8435, -1.7198, -1.5739,
        -3.6692,  2.4464], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
segment_embedding = nn.Embedding(3, EMBEDDING_DIM).to(device)

s_embedding = segment_embedding(segment_labels)
print(f"Dimensions of segment embedding: {s_embedding.size()}")# Expected: (sequence_length, batch_size, EMBEDDING_DIM)

#Check the Segment Embedding vectors for first 3 tokens of the first sample in the batch
# you get segment_embedded[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"Segment Embeddings for the {i}th token of the first sample: {s_embedding[i,0,:]}")

Dimensions of segment embedding: torch.Size([100, 3, 10])
Segment Embeddings for the 0th token of the first sample: tensor([ 0.6989, -0.1497,  1.6185,  0.6163,  0.9998,  0.5078, -0.2371,  0.4028,
        -0.2353,  1.0584], device='cuda:0', grad_fn=<SliceBackward0>)
Segment Embeddings for the 1th token of the first sample: tensor([ 0.6989, -0.1497,  1.6185,  0.6163,  0.9998,  0.5078, -0.2371,  0.4028,
        -0.2353,  1.0584], device='cuda:0', grad_fn=<SliceBackward0>)
Segment Embeddings for the 2th token of the first sample: tensor([ 0.6989, -0.1497,  1.6185,  0.6163,  0.9998,  0.5078, -0.2371,  0.4028,
        -0.2353,  1.0584], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
#Create the combined embedding vectors
bert_embeddings = t_embeddings + p_embedding + s_embedding
print(f"Dimensions of token + position + segment encoded tokens: {bert_embeddings.size()}")

#Check the BERT Embedding vectors for first 3 tokens of the first sample in the batch
# you get bert_embeddings[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"BERT_Embedding for {i}th token: {bert_embeddings[i,0,:]}")

Dimensions of token + position + segment encoded tokens: torch.Size([100, 3, 10])
BERT_Embedding for 0th token: tensor([ 2.6939,  6.3020, -3.8962,  8.3125,  6.0162,  5.1866, -1.3804, -1.3358,
        -8.4376,  1.4409], device='cuda:0', grad_fn=<SliceBackward0>)
BERT_Embedding for 1th token: tensor([ -7.4547,   5.7798, -11.2967,  -8.2675,   0.8440,   3.9294,  -0.0505,
          2.9400,  -8.0481,   5.0401], device='cuda:0',
       grad_fn=<SliceBackward0>)
BERT_Embedding for 2th token: tensor([ -4.7160,  -1.5354, -11.6607,  13.5701,   3.5465,   3.1960,  -3.6846,
         -3.7450,  -7.5749,   4.9512], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [ ]:
class BERT(torch.nn.Module):

    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        vocab_size: The size of the vocabulary.
        d_model: The size of the embeddings (hidden size).
        n_layers: The number of Transformer layers.
        heads: The number of attention heads in each Transformer layer.
        dropout: The dropout rate applied to embeddings and Transformer layers.
        """
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # Embedding layer that combines token embeddings and segment embeddings
        self.bert_embedding = BERTEmbedding(vocab_size, d_model, dropout)

        # Transformer Encoder layers
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=heads, dropout=dropout,batch_first=False)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)

        # Linear layer for Next Sentence Prediction
        self.nextsentenceprediction = nn.Linear(d_model, 2)

        # Linear layer for Masked Language Modeling
        self.masked_language = nn.Linear(d_model, vocab_size)

    def forward(self, bert_inputs, segment_labels):
        """
        bert_inputs: Input tokens.
        segment_labels: Segment IDs for distinguishing different segments in the input.
        mask: Attention mask to prevent attention to padding tokens.

        return: Predictions for next sentence task and masked language modeling task.
        """

        padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
        # Generate embeddings from input tokens and segment labels
        my_bert_embedding = self.bert_embedding(bert_inputs, segment_labels)

        # Pass embeddings through the Transformer encoder
        transformer_encoder_output = self.transformer_encoder(my_bert_embedding,src_key_padding_mask=padding_mask)


        next_sentence_prediction = self.nextsentenceprediction(transformer_encoder_output[ 0,:])


        # Masked Language Modeling: Predict all tokens in the sequence
        masked_language = self.masked_language(transformer_encoder_output)

        return  next_sentence_prediction, masked_language

In [ ]:
EMBEDDING_DIM = 10

# Define parameters
vocab_size = 147161  # Replace VOCAB_SIZE with your vocabulary size
d_model = EMBEDDING_DIM  # Replace EMBEDDING_DIM with your embedding dimension
n_layers = 2  # Number of Transformer layers
initial_heads = 12 # Initial number of attention heads
initial_heads = 2
# Ensure the number of heads is a factor of the embedding dimension
heads = initial_heads - d_model % initial_heads

dropout = 0.1  # Dropout rate

# Create an instance of the BERT model
model = BERT(vocab_size, d_model, n_layers, heads, dropout).to(device)

In [ ]:
padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
padding_mask.shape

torch.Size([3, 100])

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=heads, dropout=dropout,batch_first=False).to(device)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers).to(device)
# Pass embeddings through the Transformer encoder
transformer_encoder_output = transformer_encoder(bert_embeddings,src_key_padding_mask=padding_mask)
transformer_encoder_output.shape

torch.Size([100, 3, 10])

In [ ]:
nextsentenceprediction = nn.Linear(d_model, 2).to(device)
nsp = nextsentenceprediction(transformer_encoder_output[ 0,:])
#logits for NSP task
print(f"NSP Output Shape: {nsp.shape}")  # Expected shape: (batch_size, 2)

NSP Output Shape: torch.Size([3, 2])


In [ ]:
masked_language = nn.Linear(d_model, vocab_size).to(device)
# Masked Language Modeling: Predict all tokens in the sequence
mlm = masked_language(transformer_encoder_output)
#logits for MLM task
print(f"MLM Output Shape: {mlm.shape}")  # Expected shape: (seq_length, batch_size, vocab_size)

MLM Output Shape: torch.Size([100, 3, 147161])


## Evaluation

After creating the BERT model, the next step is training and evaluating its performance. To facilitate this, an `evaluate` function is defined with the following steps:

1. Loss Function: The CrossEntropyLoss function is defined to calculate the loss between predicted and actual values.

2. Function Arguments: The function takes arguments including the dataloader, model, loss function, and device.

3. Evaluation Mode: The BERT model is put into evaluation mode using `model.eval()`, disabling dropout and training-specific behaviors. Variables are initialized to track the total loss, total next sentence loss, total mask loss, and total number of batches.

4. Evaluation Loop: The function iterates through the batches in the provided dataloader.

5. Forward Pass: A forward pass is performed with the BERT model to obtain predictions for the next sentence and masked language tasks.

6. Loss Calculation: The losses for the next sentence and masked language tasks are calculated, and then summed up to obtain the total loss for the batch.

7. Average Loss Calculation: The average loss, average next sentence loss, and average mask loss are calculated by dividing the total losses by the total number of batches.


The `evaluate` function is used not only for evaluating the BERT model's performance but also during the training phase to assess the model's progress.


In [ ]:
PAD_IDX=0
loss_fn_mlm = nn.CrossEntropyLoss(ignore_index=PAD_IDX)# The loss function must ignore PAD tokens and only calculates loss for the masked tokens
loss_fn_nsp = nn.CrossEntropyLoss()

In [ ]:
def evaluate(dataloader=test_dataloader, model=model, loss_fn_mlm=loss_fn_mlm, loss_fn_nsp=loss_fn_nsp, device=device):
    model.eval()  # Turn off dropout and other training-specific behaviors

    total_loss = 0
    total_next_sentence_loss = 0
    total_mask_loss = 0
    total_batches = 0
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        for batch in dataloader:
            bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]

            # Forward pass
            next_sentence_prediction, masked_language = model(bert_inputs, segment_labels)

            # Calculate loss for next sentence prediction
            # Ensure is_nexts is of the correct shape for CrossEntropyLoss
            next_loss = loss_fn_nsp(next_sentence_prediction, is_nexts.view(-1))

            # Calculate loss for predicting masked tokens
            # Flatten both masked_language predictions and bert_labels to match CrossEntropyLoss input requirements
            mask_loss = loss_fn_mlm(masked_language.view(-1, masked_language.size(-1)), bert_labels.view(-1))

            # Sum up the two losses
            loss = next_loss + mask_loss
            if torch.isnan(loss):
                continue
            else:
                total_loss += loss.item()
                total_next_sentence_loss += next_loss.item()
                total_mask_loss += mask_loss.item()
                total_batches += 1

    avg_loss = total_loss / (total_batches + 1)
    avg_next_sentence_loss = total_next_sentence_loss / (total_batches + 1)
    avg_mask_loss = total_mask_loss / (total_batches + 1)

    print(f"Average Loss: {avg_loss:.4f}, Average Next Sentence Loss: {avg_next_sentence_loss:.4f}, Average Mask Loss: {avg_mask_loss:.4f}")
    return avg_loss

## Training
The training process for the BERT model involves the following steps:

1. Optimizer Definition: Before training starts, an optimizer is defined for training the BERT model. In this case, the Adam optimizer is used.

2. Training Loop: Within each epoch, the training data is iterated through in batches.

3. Forward Pass: For each batch, a forward pass is performed, where the BERT model predicts the next sentence and masked language tasks.

4. Loss Calculation and Parameter Update: The loss is calculated based on the predicted and actual values. The model's parameters are then updated through backpropagation and gradient clipping.

5. Epoch Evaluation: After each epoch, the average training loss is printed. The model's performance on the test set is evaluated. Additionally, the model is saved after each epoch.

These steps are repeated for multiple epochs to train the BERT model and monitor its progress over time.

**NOTE: The current DataLoader is quite huge, and it will take several hours for the model to train with such a huge dataset. Hence, below is the randomly sampled dataset (which is relatively small which still takes 1-2 hours to run) from IMDB to make the process faster. If you want to train the model on the entire dataset, skip the next cell and directly run the training cell.**


In [ ]:
BATCH_SIZE = 3

train_dataset_path = './bert_dataset/bert_train_data_sampled.csv'
test_dataset_path = './bert_dataset/bert_test_data_sampled.csv'

train_dataset = BERTCSVDataset(train_dataset_path)
test_dataset = BERTCSVDataset(test_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [115]:
# Define the optimizer
optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=0.01, betas=(0.9, 0.999))

# Training loop setup
num_epochs = 1
total_steps = num_epochs * len(train_dataloader)

# Define the number of warmup steps, e.g., 10% of total
warmup_steps = int(total_steps * 0.1)

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

# Lists to store losses for plotting
train_losses = []
eval_losses = []

for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]

        optimizer.zero_grad()
        next_sentence_prediction, masked_language = model(bert_inputs, segment_labels)

        next_loss = loss_fn_nsp(next_sentence_prediction, is_nexts)
        mask_loss = loss_fn_mlm(masked_language.view(-1, masked_language.size(-1)), bert_labels.view(-1))

        loss = next_loss + mask_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()  # Update the learning rate

        total_loss += loss.item()

        if torch.isnan(loss):
            continue
        else:
            total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader) + 1
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch+1} - Average training loss: {avg_train_loss:.4f}")

    # Evaluation after each epoch
    eval_loss = evaluate(test_dataloader, model, loss_fn_nsp, loss_fn_mlm, device)
    eval_losses.append(eval_loss)

Epoch 1: 100%|██████████| 3334/3334 [00:56<00:00, 58.95it/s]


Epoch 1 - Average training loss: 24.9981


Training Epochs: 100%|██████████| 1/1 [01:07<00:00, 67.12s/it]

Average Loss: 12.5464, Average Next Sentence Loss: 0.7335, Average Mask Loss: 11.8129


To evaluate the performance of a pretrained BERT model in predicting whether a second sentence follows the first, a function called `predict_nsp` is defined. The function operates as follows:

1. Tokenization: The input sentences are tokenized using the `tokenizer.encode_plus` method, which returns a dictionary of tokenized inputs. These tokenized inputs are then converted into tensors and moved to the appropriate device for processing.

2. Prediction: The BERT model is utilized to make predictions by passing the token and segment tensors as input.

3. Logits Manipulation: The first element of the logits tensor is selected and unsqueezed to add an extra dimension, resulting in a shape of `[1, 2]`.

4. Probability and Prediction: The logits are passed through a softmax function to obtain probabilities, and the prediction is obtained by taking the argmax.

5. Result Interpretation: The prediction is interpreted and returned as a string, indicating whether the second sentence follows the first or not.

6. Example Usage: An example usage of the `predict_nsp` function is provided, demonstrating how two example sentences can be passed to the function along with a BERT model and tokenizer. The result is printed, indicating whether the second sentence follows the first based on the model's prediction.

By utilizing the `predict_nsp` function, the performance of the pretrained BERT model in determining the relationship between two sentences can be assessed.


In [112]:
# Initialize the tokenizer with the BERT model's vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.eval()

def predict_nsp(sentence1, sentence2, model, tokenizer):
    # Tokenize sentences with special tokens
    tokens = tokenizer.encode_plus(sentence1, sentence2, return_tensors="pt")
    tokens_tensor = tokens["input_ids"].to(device)
    segment_tensor = tokens["token_type_ids"].to(device)

    # Predict
    with torch.no_grad():
        # Assuming the model returns NSP predictions first
        nsp_prediction, _ = model(tokens_tensor, segment_tensor)
        # Select the first element (first sequence) of the logits tensor
        first_logits = nsp_prediction[0].unsqueeze(0)  # Adds an extra dimension, making it [1, 2]
        logits = torch.softmax(first_logits, dim=1)
        prediction = torch.argmax(logits, dim=1).item()

    # Interpret the prediction
    return "Second sentence follows the first" if prediction == 1 else "Second sentence does not follow the first"

# Example usage
sentence1 = "The cat sat on the mat."
sentence2 = "It was a sunny day"

print(predict_nsp(sentence1, sentence2, model, tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Second sentence does not follow the first


A function is defined to perform Masked Language Modeling (MLM) using a pretrained BERT model. The function operates as follows:

1. Tokenization: The input sentence is tokenized using the tokenizer and converted into token IDs, including the special tokens. The tokenized sentence is stored in the `tokens_tensor` variable.

2. Segment Labels: Dummy segment labels filled with zeros are created and stored as `segment_labels`.

3. Prediction: The BERT model is used to make predictions by passing the token tensor and segment labels as input. The MLM logits are extracted as `predictions`.

4. Mask Token Index: The position of the [MASK] token is identified using the `nonzero` method and stored in the `mask_token_index` variable. Note that all tokens except the mask token are zero-padded.

5. Predicted Index: The predicted index for the [MASK] token is obtained by taking the argmax of the MLM logits at the corresponding position.

6. Token Conversion: The predicted index is converted back to a token using the `convert_ids_to_tokens` method of the tokenizer.

7. Replaced Sentence: The original sentence is replaced with the predicted token at the position of the [MASK] token, resulting in the predicted sentence.


In [122]:
def predict_mlm(sentence, model, tokenizer):
    # Tokenize the input sentence and convert to token IDs, including special tokens
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens_tensor = inputs.input_ids.to(device)

    # Create dummy segment labels filled with zeros, assuming it's needed by your model
    segment_labels = torch.zeros_like(tokens_tensor).to(device)

    with torch.no_grad():
        # Forward pass through the model, now correctly handling the output tuple
        output_tuple = model(tokens_tensor, segment_labels)

        # Assuming the second element of the tuple contains the MLM logits
        predictions = output_tuple[1]  # Adjusted based on your model's output

        # Identify the position of the [MASK] token
        mask_token_index = (tokens_tensor == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        # Get the predicted index for the [MASK] token from the MLM logits
        predicted_index = torch.argmax(predictions[0, mask_token_index.item(), :], dim=-1)
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index.item()])[0]

        # Replace [MASK] in the original sentence with the predicted token
        predicted_sentence = sentence.replace(tokenizer.mask_token, predicted_token, 1)

    return predicted_sentence


# Example usage
sentence = "The cat sat on the [MASK]."
print(predict_mlm(sentence, model, tokenizer))

The cat sat on the [unused4].
